# Setup

In [17]:
import os
import shutil
from datetime import datetime

## Configuration

In [2]:
WORKING_DIR = "/kaggle/working/"
CODE_DIR = "/kaggle/temp/src"
GDRIVE_DIR = "/content/drive/MyDrive/AIO25-MIX002/Projects/AIO2025_Conquer_CONQ008_M7_Project/"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download code

In [18]:
# ! Change branch name if not main
BRANCH = "main"

In [19]:
# If directory "src" not exist then clone a new one
!pwd
![ -d "{CODE_DIR}" ] || git clone --depth 1  --branch "{BRANCH}" "https://github.com/aio25-mix002/m07-p7.1" "{CODE_DIR}"


/kaggle/temp/src


## Fetch the latest code

In [20]:
# Go to CODE_DIR, Fetch the latest code
%cd {CODE_DIR}
!git clean -fdx
!git status
!git pull
!pwd


/kaggle/temp/src
Removing src/__pycache__/
On branch ngocdung/make-inference-n-submit
Your branch is up to date with 'origin/ngocdung/make-inference-n-submit'.

nothing to commit, working tree clean
Already up to date.
/kaggle/temp/src


# Data Preparation

### Option 1: Using Kaggle API Credentials (Only if data in Google Drive not available)

First, ensure you have downloaded your `kaggle.json` API token from your Kaggle account. Once downloaded, upload it to your Colab session. You can do this via the 'Files' tab on the left sidebar.

After uploading, we will move it to the correct directory (`~/.kaggle/`) and set the necessary permissions.

In [21]:
import os

# Create the .kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle

# Move the uploaded kaggle.json file to the .kaggle directory
# Assuming kaggle.json is in the current working directory after upload
# If you uploaded it to a different path, please adjust '/content/kaggle.json'
!mv /content/kaggle.json ~/.kaggle/kaggle.json

# Set permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

print('Kaggle API credentials set up successfully!')
!ls -la ~/.kaggle/kaggle.json

Kaggle API credentials set up successfully!
-rw------- 1 root root 67 Jan  9 07:52 /root/.kaggle/kaggle.json


In [22]:
# !python download_data.py
!kaggle competitions download -c action-video
!unzip -q action-video.zip  -d {WORKING_DIR}

 97% 3.06G/3.14G [00:10<00:00, 161MB/s] 
100% 3.14G/3.14G [00:10<00:00, 316MB/s]


### (Unused due to synchronizing problem) Option 2: Using data saved in Google Drive


In [9]:
# import os
# import shutil

# source_gdrive_data_path = os.path.join(GDRIVE_DIR, 'Data')
# destination_working_data_path = os.path.join(WORKING_DIR, 'data')

# print(f"Attempting to copy data from Google Drive: {source_gdrive_data_path}")
# print(f"To working directory: {destination_working_data_path}")

# try:
#     # Create the destination directory if it doesn't exist. If it exists, remove it first to avoid errors.
#     if os.path.exists(destination_working_data_path):
#         print(f"Destination directory {destination_working_data_path} already exists. Removing before copy...")
#         shutil.rmtree(destination_working_data_path)

#     shutil.copytree(source_gdrive_data_path, destination_working_data_path)
#     print(f"Successfully copied '{source_gdrive_data_path}' to '{destination_working_data_path}'")
# except FileNotFoundError:
#     print(f"Error: Source data directory not found in Google Drive at {source_gdrive_data_path}")
# except Exception as e:
#     print(f"An error occurred while copying the data from Google Drive: {e}")

# Train

### Training

In [28]:
# Training - can change number of epochs
!python train.py #--epochs 1

Using device: cuda
Initializing datasets...
Train size: 5628 | Val size: 626
Creating model...
model.safetensors: 100% 346M/346M [00:00<00:00, 360MB/s]
Loaded pretrained weights. Missing: 132, Unexpected: 0
🚀 Compiling model with torch.compile...

Training Configuration:
  Epochs: 1
  Batch size: 8
  Learning rate: 0.0001
  Num frames: 16
  Frame stride: 2
  Val ratio: 0.1
  Checkpoint dir: ./checkpoints

Backbone FROZEN (Chỉ train SMIF & Head)

Epoch 1/1
Train:   0% 0/704 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensor

### Save the best checkpoint to Google Drive

In [29]:
import os
import shutil
from datetime import datetime

# Define the brief note for the filename
NOTE = "vanilla" # !!! Should edit every new run
SAVE_PATH = f"{GDRIVE_DIR}Artifacts/Checkpoints"


In [30]:
# Define the source path of the best model checkpoint
source_checkpoint_path = os.path.join(CODE_DIR, 'checkpoints', 'best_model.pth')

# Define your Google Drive destination folder path
# IMPORTANT: Please replace 'YOUR_GOOGLE_DRIVE_FOLDER_PATH' with the actual path to your folder in Google Drive.
# For example, it might be '/content/drive/MyDrive/MyProjectCheckpoints/'

# Ensure the Google Drive folder exists
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
    print(f"Created Google Drive folder: {SAVE_PATH}")

# Generate a timestamp for the filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Construct the new filename for the checkpoint
new_checkpoint_filename = f"model_{timestamp}_{NOTE}.pth"
destination_checkpoint_path = os.path.join(SAVE_PATH, new_checkpoint_filename)

# Copy the checkpoint
try:
    shutil.copy(source_checkpoint_path, destination_checkpoint_path)
    print(f"Checkpoint successfully saved to: {destination_checkpoint_path}")
except FileNotFoundError:
    print(f"Error: Source checkpoint not found at {source_checkpoint_path}")
except Exception as e:
    print(f"An error occurred while saving the checkpoint: {e}")


Checkpoint successfully saved to: /content/drive/MyDrive/AIO25-MIX002/Projects/AIO2025_Conquer_CONQ008_M7_Project/Artifacts/Checkpoints/model_20260109_082113_vanilla.pth


### Reload the checkpoint (if needed)

In [36]:
import torch
import os

# # Define the path to the checkpoint file
destination_checkpoint_path = source_checkpoint_path #f'{GDRIVE_DIR}Artifacts/Checkpoints/model_20260107_081244_vanilla.pth'

# Check if the checkpoint file exists
if os.path.exists(destination_checkpoint_path):
    # Load the checkpoint
    loaded_checkpoint = torch.load(destination_checkpoint_path, map_location=torch.device('cpu')) # Use 'cuda' if you want to load to GPU
    print(f"Checkpoint loaded successfully from: {destination_checkpoint_path}")
    print("Keys in the loaded checkpoint:", loaded_checkpoint.keys())

    # Example of how you might load it into a model (assuming 'model' is defined)
    # model.load_state_dict(loaded_checkpoint['model_state_dict'])
    # optimizer.load_state_dict(loaded_checkpoint['optimizer_state_dict'])
    # epoch = loaded_checkpoint['epoch']
    # loss = loaded_checkpoint['loss']
else:
    print(f"Error: Checkpoint not found at {destination_checkpoint_path}")


Checkpoint loaded successfully from: /kaggle/temp/src/checkpoints/best_model.pth
Keys in the loaded checkpoint: odict_keys(['_orig_mod.smif.alpha', '_orig_mod.smif.conv_fuse.weight', '_orig_mod.smif.conv_fuse.bias', '_orig_mod.backbone.cls_token', '_orig_mod.backbone.pos_embed', '_orig_mod.backbone.patch_embed.proj.weight', '_orig_mod.backbone.patch_embed.proj.bias', '_orig_mod.backbone.blocks.0.norm1.weight', '_orig_mod.backbone.blocks.0.norm1.bias', '_orig_mod.backbone.blocks.0.attn.qkv.weight', '_orig_mod.backbone.blocks.0.attn.qkv.bias', '_orig_mod.backbone.blocks.0.attn.proj.weight', '_orig_mod.backbone.blocks.0.attn.proj.bias', '_orig_mod.backbone.blocks.0.norm2.weight', '_orig_mod.backbone.blocks.0.norm2.bias', '_orig_mod.backbone.blocks.0.mlp.fc1.weight', '_orig_mod.backbone.blocks.0.mlp.fc1.bias', '_orig_mod.backbone.blocks.0.mlp.fc2.weight', '_orig_mod.backbone.blocks.0.mlp.fc2.bias', '_orig_mod.backbone.blocks.0.lmim.delta', '_orig_mod.backbone.blocks.0.lmim.reduce.weight', 

# Submission

### Make submission file

In [38]:
destination_checkpoint_path

'/kaggle/temp/src/checkpoints/best_model.pth'

In [39]:

!python inference.py --checkpoint {destination_checkpoint_path} \
    --data_root {WORKING_DIR}data/test




Using device: cuda
INFERENCE ON TEST SET
Loading checkpoint from /kaggle/temp/src/checkpoints/best_model.pth...
Model loaded

Loading test dataset...
Test samples: 510

Running inference...
Processed 160/510 samples
Processed 320/510 samples
Processed 480/510 samples

Inference complete! Processed 510 videos

✓ Submission file created at: /kaggle/working/submission.csv

Submission saved to: /kaggle/working/submission.csv


### Submit to Kaggle

In [40]:
# !Must specify message
MESSAGE = "Testing runbook.ipynb 3rd time"

In [41]:
!kaggle competitions submit -c action-video -f /kaggle/working/submission.csv -m "{MESSAGE}"

100% 3.23k/3.23k [00:00<00:00, 20.0kB/s]
Successfully submitted to AIO-2025: Video Action Classification Challenge